# Vinculación de términos

La vinculación de términos es un proceso que identifica similitudes entre dos términos que pueden ayudar a relacionarlos dentro de una búsqueda.

La vinculación, generalmente, responde a aspectos semánticos de los términos. Por ejemplo, se pueden vincular sinónimos, hiperónimos, etc. Por tanto, la computadora debe tener información acerca de características del significado de palabras.

Aquí proponemos tres formas de vinculación:

* Vinculación de Lin
* Vinculación por precisión media
* Vinculación dirigida

Usamos información obtenida del lexicón <b>WordNet</b> (<u>https://wordnet.princeton.edu/</u>) para obtener rasgos semánticos de los términos.

In [1]:
import numpy as np
import nltk
from nltk.corpus import wordnet
from itertools import chain

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Vinculación de Lin

En base a los rasgos que pueden obtenerse de este lexicón, la vinculación de LIN estima la similitud entre dos términos en base a la siguiente formulación:

$$LIN(w,u) = \frac{common(w,u)}{union(w,u)}$$

Donde $common$ refiere a la intersección de rasgos y $union$ a la unión del conjunto de rasgos en ambos términos.

Una versión logarítmica de la vinculación de LIN esta determinada como:

$$LIN(w,u) = \frac{\log_2 common(w,u)}{\log_2 difference(w,u)}$$

Donde $difference$ hace referencia a los rasgos diferenciales de ambos términos.

In [2]:
def log2(x):
  """
  Función para regresar logaritmo en base 2.
  Considera el log(0) como 0
  """
  if x > 0:
    return np.log2(x)
  else:
    return 0

#Función para extraer los rasgos de los términos
features = lambda term: set(chain(*[w.lemma_names() for w in wordnet.synset(term + '.n.01').hypernyms()]))
#Función de intersección de rasgos entre dos términos
common = lambda t1,t2: features(t1).intersection(features(t2))
#Función de diferencia de rasgos entre dos términos>
description = lambda t1,t2: features(t1).difference(features(t2))

def LIN(t1,t2,log=False):
  """
  Función para calcular la vinculación de LIN.

  Arguments
  ---------
  t1, t2 : str
    Términos que se van a vincular entre sí
  log : bool
    Indica si el calculo se hará con valores logarítmicos o no

  Returns
  -------
    Valor de vinculación en base a Lin
  """
  #Calcula rasgos en común
  p_comm = len(common(t1,t2))
  #Calcula rasgos diferenciales
  p_desc = len(description(t1,t2))

  #Regresa el calculo de Lin
  #en diferentes circunstancias
  if log:
    if log2(p_desc) == 0:
      return log2(p_comm)
    else:
      return log2(p_comm)/log2(p_desc)
  else:
    return p_comm/(len(features(t1))+len(features(t2)))

In [ ]:
#Ejemplos de vinculación
print(LIN('domestic_cat','car'))
print(LIN('domestic_cat','domestic_dog'))
print(LIN('domestic_cat','Felis_domesticus'))

0.0
0.25
0.5


## Vinculación por precisión media

La vinculación de términos por precisión media es una forma de estimar una vinculación dirigida (jerárquica) que se basa en una función de precisión:

$$P(r;w,u) = \frac{|\{\phi_i : w_i \cap u_i, i=1,...,r\}|}{r}$$

Además, se define una función de relevancia sobre los rasgos:

$$rel'(\phi) = \begin{cases} 1-\frac{rank(\phi,w)}{|w|+1} & \text{ si } \phi\in w \\ 0 & \text{ en otro cas} \end{cases}$$

Finalmente, la precisión media se determina como:

$$APinc(w\to u) = \frac{1}{|w|}\sum_r P(r;w,u) rel'(\phi_r(w))$$

In [3]:
def relevance(feature, term, rank=True):
  """
  Función de relación para rasgos.

  Arguments
  ---------
  feature : str
    Rasgo que se va a considerar
  term : str
    Término dentro del cual se va a buscar el rasgo

  Returns
  -------
    Valor de relación del rasgo en el término
  """
  feats = features(term)
  if rank:
    if feature in feats:
      rank = [r+1 for r,f in enumerate(feats) if f == feature][0]/len(feats)
      return 1 - (rank/(len(feats)+1))
    else:
      return 0
  else:
    return 1*(feature in feats)

def precision(t_parent, t_child, r):
  """
  Función para calcular valores de precision entre términos.

  Arguments
  ---------
  t_parent : str
    Término considerado candidato a padre del segundo término
  t_child : str
    Término considerado candidato a hijo del otro término
  r : int
    Valor de los features más relevantes de término padre

  Returns
  -------
  Valor de precisión entre ambos términos
  """
  #Features en común
  feat_parent = list(features(t_parent))[:r]
  comm = set(feat_parent).intersection(features(t_child))
  #Valor calculado
  p = len(comm)/r

  return p

def APinc(parent, child):
  """
  Función para calcular la vinculación de precisión media APinc.

  Arguments
  --------
  parent : str
    Término considerado candidato a padre del segundo término
  child : str
    Término considerado candidato a hijo del otro término

  Returns
  -------
  Valor de vinculación de precisión media
  """
  #Almacena el valor APinc
  mean_prec = 0
  #Corre en número de features
  i = 1
  for feature in features(parent):
    #Calcula la precisión media
    mean_prec += precision(parent,child,i)*relevance(feature,parent)
    #Avanza a la siguiente feature
    i += 1

  return mean_prec/i

In [4]:
print(APinc('domestic_dog','domestic_cat'))
print(APinc('Felis_domesticus', 'domestic_cat'))
print(APinc('Felis_domesticus','car'))

0.22666666666666666
0.7
0.0


## Vinculación dirigida

Una versión de la vinculación dirigida que pondera el $APinc$ con la vinculación de Lin se define como:

$$balAPinc(w\to u) = \sqrt{LIN(w,u) APinc(w \to u)}$$

In [5]:
def balAPinc(t1,t2, log=False):
  """
  Función para calcular el valor del balance APinc.

  Arguments
  ---------
  t1, t2 : str
    Términos para vincular
  log : bool
    Indica si la medida de LIN usará valores logarítmicos

  Returns
  -------
    Valor de balAPinc que vincula ambos términos
  """
  #Calcula Lin
  lin = LIN(t1,t2,log=log)
  #Calcula APinc
  apinc = APinc(t1,t2)

  return np.sqrt(lin*apinc)

In [6]:
print(balAPinc('domestic_cat','domestic_dog'))
print(balAPinc('domestic_cat','Felis_domesticus'))
print(balAPinc('Felis_domesticus','car'))

0.1848422751068236
0.5916079783099616
0.0
